In [1]:
import pandas as pd

In [2]:
from amplpy import AMPL,Environment
ampl = AMPL(Environment('/home/asino/Downloads/ampl_linux-intel64/'))
ampl.setOption("solver","/home/asino/Downloads/ampl_linux-intel64/cplex")
ampl.setOption("absmipgap",1)



In [3]:
ME="me"

In [4]:
def fillDictOfBrokenPlayers():
    bought=pd.read_csv("availability-of-players.csv")
    dictOfMyPlayers={}
    dictOfBrokenPlayers={}
    for idx,player in bought.iterrows():
        if player["Owner"]==ME:
            if player["CanPlay"]==0:
                dictOfBrokenPlayers[player["Name"]]=1
            else:
                dictOfBrokenPlayers[player["Name"]]=0
    return dictOfBrokenPlayers

dictOfBrokenPlayers=fillDictOfBrokenPlayers()
print(dictOfBrokenPlayers)   

{'CRISCITO': 1, 'STRAKOSHA': 0, 'ZIELINSKI': 0, 'KURTIC': 0, 'ZAPATAD': 0, 'IZZO': 0, 'NKOULOU': 1, 'KOULIBALY': 0, 'POLITANO': 1, 'KESSIE': 0, 'BONIFAZI': 0, 'MURRU': 0, 'RADU': 0, 'QUAGLIARELLA': 0, 'VERETOUT': 0, 'GOMEZA': 0, 'BONUCCI': 0, 'BERISHAE': 0, 'FLORENZI': 0, 'MARTINEZL': 0, 'CHIELLINI': 0, 'MILENKOVIC': 0, 'SUSO': 0, 'MANDRAGORA': 0, 'NANDEZ': 0, 'IAGOFALQUE': 0, 'BALOTELLI': 0, 'PROTO': 0}


In [5]:
ampl.read("07-prepare-selection.mod")
ampl.readData("operations-mantra-online.dat")

	07-prepare-selection.mod
	line 33 offset 991
	Caution: 0-dimensional slice


In [6]:
def suggestTeam(dictOfBrokenPlayers):
    plas=ampl.getSet("Players").getValues()
    valuesForBrokenPlayersAmpl={}
    for name in plas:
        name=name[0]
        #print(name)
        valuesForBrokenPlayersAmpl[name]=0
    for player,value in dictOfBrokenPlayers.items():
        valuesForBrokenPlayersAmpl[player]=value
    ampl.getParameter("broken").setValues(valuesForBrokenPlayersAmpl)
        
        
        
    
    ampl.solve()


In [7]:
def analyseSolution():
    putPlayerInSlot=ampl.getVariable("putPlayerInSlot")
    cost=ampl.getParameter("cost")
    pfs=ampl.getParameter("performance")
    moduleChosen=ampl.getVariable("bidderChoosesModule")
    globalPerf=ampl.getVariable("minPerf").value()
    
    roles=ampl.getSet("CompatibilityPlayerRole").getValues().toDict()
    slots=ampl.getSet("CompatibilitySlotRole").getValues().toDict()
    solution=putPlayerInSlot.getValues().toDict()
    costs=cost.getValues().toDict()
    pfs=pfs.getValues().toDict()
    modules=moduleChosen.getValues().toDict()
    moduleUsed="AAA"
    

    for moduleName,taken in modules.items():
        if taken>0.5:
            #print("Module: {}\n\n".format(moduleName))
            moduleUsed=moduleName
    myPls={}
    for key,val in solution.items():

        if key[0]==moduleUsed:
            if val>0.5:
                myPls[key[2]]=key[1]
                #print(key,val)
    rolesA={}

    #To show the roles of each player
    for key,val in roles.items():
        if key[0] not in rolesA.keys():
            rolesA[key[0]]=[]
        rolesA[key[0]].append(key[1])


    mappingSlotNumberToPossibleRoles={}
    for key,val in slots.items():
        if key[0]==moduleUsed:

            slotNr=key[1]
            role=key[2]
            if slotNr not in mappingSlotNumberToPossibleRoles.keys():
                mappingSlotNumberToPossibleRoles[slotNr]=[]
            mappingSlotNumberToPossibleRoles[slotNr].append(role)

    emptySlotsInField=0
    emptySlotsOutside=0
    message=""
    for slotNr,possibleRoles in mappingSlotNumberToPossibleRoles.items():
        if slotNr<=11:
            privilege="TOP"
        else:
            privilege="NO"
        try:
            message+=("{} plays  in slot {}  {}\n".format(myPls[slotNr], possibleRoles,privilege))
        except KeyError:
            if privilege=="TOP":
                emptySlotsInField+=1
            else:
                emptySlotsOutside+=1
    return globalPerf,moduleUsed,message,emptySlotsInField,emptySlotsOutside
    

In [8]:
pls=dictOfBrokenPlayers.keys()
dictOfEffects={}
solidPlayers=["STRAKOSHA","BERISHAE","IZZO"]
for player in pls:
    for p2 in pls:
        if player >p2:
            if player not in solidPlayers and p2 not in solidPlayers:
                broks=fillDictOfBrokenPlayers()
                broks[player]=1
                broks[p2]=1
                suggestTeam(broks)
                dictOfEffects[(player,p2)]=(analyseSolution())
#suggestTeam({},{})

CPLEX 12.8.0.0: optimal integer solution; objective 84.16829957
1417 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.01535457
1215 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607
1795 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607
1552 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.06465088
1434 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.54383998
1574 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 80.99457224
1349 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.06465088
1534 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 80.64645582


CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607
1857 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.39351246
1408 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.94833533
1101 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.00970198
1025 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.94833533
1327 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607
1451 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.00377562
1515 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 80.99487687
1307 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.51592428


CPLEX 12.8.0.0: optimal integer solution; objective 81.55265198
1166 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.49128533
1174 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.86446607
1302 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.86446607
1396 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.70206466
1561 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.86943966
1473 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.70206466
1554 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.38432422
1646 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.70206466


CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607
1447 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.39351246
1408 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.56088746
1267 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 80.74113717
1233 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.38461371
1350 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.90566112
1376 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 80.71373545
1396 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.75271612
1390 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.73745296


CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607
2026 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.41307194
1357 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.16829957
1447 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 81.87604502
1034 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.57839745
1412 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.01535457
1501 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 83.60211496
1103 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607
1542 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 84.32151607


In [9]:
couples=dictOfEffects.keys()
cp=sorted(couples,key=lambda k:dictOfEffects[k][0])
for i in range(10):
    print(cp[i],dictOfEffects[cp[i]][0])
    #print(dictOfEffects[cp[i]][2])
    #break

('RADU', 'MURRU') 77.63920291107843
('SUSO', 'GOMEZA') 79.03055797332904
('ZIELINSKI', 'SUSO') 79.2090291868347
('SUSO', 'MANDRAGORA') 79.40049585851581
('SUSO', 'IAGOFALQUE') 79.40049585851581
('SUSO', 'MURRU') 79.46267258157154
('SUSO', 'KESSIE') 79.55745023946628
('ZIELINSKI', 'MURRU') 79.61917983220003
('MURRU', 'KESSIE') 79.96760088483161
('VERETOUT', 'SUSO') 80.03560813420312
